In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import joblib
import os
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from scipy.sparse import hstack, csr_matrix

# Mount Google Drive
drive.mount('/content/drive')

# ----------- Đọc dữ liệu --------------------
def load_data(filepath: str):
    if filepath.endswith(".xlsx"):
        return pd.read_excel(filepath)
    elif filepath.endswith(".parquet"):
        return pd.read_parquet(filepath)
    else:
        raise ValueError("Unsupported file type")

# ----------- Phân nhóm ngành ----------------
def classify_sector(text, group):
    text = str(text).lower().strip()
    for keyword, main_group in group.items():
        if keyword in text:
            return main_group
    return "Ngành khác"

def apply_sector_mapping(df, column_name, group):
    df["nganh_nghe_chuan_hoa"] = df[column_name].apply(lambda x: classify_sector(x, group))
    return df

# ----------- Vector hoá kỹ năng WORD2VEC ----------------
def tokenize(text):
    return text.lower().split()

def vectorize_skills_word2vec(df, col="ky_nang_combined", model_path="word2vec_model.model"):
    sentences = df[col].fillna("").apply(tokenize).tolist()
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    model.save(model_path)

    def sentence_vector(sentence):
        vecs = [model.wv[word] for word in sentence if word in model.wv]
        return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

    X_vectors = np.array([sentence_vector(sent) for sent in sentences])
    return X_vectors

# ----------- Encode categorical (OHE) ----------------
def group_experience(x):
    if x <= 1: return "0-1"
    elif x <= 3: return "1-3"
    elif x <= 5: return "3-5"
    elif x <= 10: return "5-10"
    else: return ">10"

def encode_categorical(df, cols, model_path="ohe_encoder.pkl"):
    encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
    X_cat = encoder.fit_transform(df[cols])
    with open(model_path, "wb") as f:
        pickle.dump(encoder, f)
    return X_cat

# ----------- MultiLabel ngành nghề ----------------
def multilabel_nganh(df, column, mlb_path="mlb_nganh.pkl"):
    df['nganh_nghe_list'] = df[column].apply(lambda x: [x.strip()] if isinstance(x, str) else [])
    mlb = MultiLabelBinarizer()
    X_nganh = mlb.fit_transform(df['nganh_nghe_list'])
    joblib.dump(mlb, mlb_path)
    return X_nganh

# ----------- Chuẩn bị ----------------
def main():
    df = load_data("/content/drive/MyDrive/CAPSTONE PROJECT/jobs_clean_final.parquet")

    # Gán nhóm ngành
    group = {
        "bán hàng": "Kinh doanh",
        "bán hàng kỹ thuật": "Kinh doanh",
        "kinh doanh": "Kinh doanh",
        "đối ngoại": "Kinh doanh",
        "tiếp thị": "Kinh doanh",
        "marketing": "Kinh doanh",
        "kế toán": "Kế toán / Kiểm toán",
        "kiểm toán": "Kế toán / Kiểm toán",
        "kiến trúc": "Kiến trúc / Xây dựng",
        "xây dựng": "Kiến trúc / Xây dựng",
        "cntt - phần mềm": "Công nghệ thông tin / Viễn thông",
        "cntt - phần cứng": "Công nghệ thông tin / Viễn thông",
        "mạng": "Công nghệ thông tin / Viễn thông",
        "viễn thông": "Công nghệ thông tin / Viễn thông",
        "phân tích hệ thống": "Công nghệ thông tin / Viễn thông",
        "sản xuất": "Sản xuất",
        "ô tô": "Sản xuất",
        "sửa chữa": "Sản xuất",
        "tự động hóa": "Sản xuất",
        "vận hành sản xuất": "Sản xuất",
        "cơ khí": "Khoa học & Kỹ thuật",
        "điện công nghiệp": "Khoa học & Kỹ thuật",
        "kỹ thuật": "Khoa học & Kỹ thuật",
        "thống kê": "Khoa học & Kỹ thuật",
        "công nghệ sinh học": "Khoa học & Kỹ thuật",
        "tiếp thị trực tuyến": "Tiếp thị, Quảng cáo / Truyền thông",
        "nghệ thuật": "Tiếp thị, Quảng cáo / Truyền thông",
        "giải trí": "Tiếp thị, Quảng cáo / Truyền thông",
        "vận chuyển": "Hậu cần / Xuất nhập khẩu / Kho bãi",
        "xuất nhập khẩu": "Hậu cần / Xuất nhập khẩu / Kho bãi",
        "kho vận": "Hậu cần / Xuất nhập khẩu / Kho bãi",
        "hàng gia dụng": "Hậu cần / Xuất nhập khẩu / Kho bãi",
        "hành chính": "Hành chính văn phòng",
        "thư ký": "Hành chính văn phòng",
        "ngân hàng": "Ngân hàng & Dịch vụ tài chính",
        "chứng khoán": "Ngân hàng & Dịch vụ tài chính",
        "bảo hiểm": "Ngân hàng & Dịch vụ tài chính",
        "nhân sự": "Nhân sự / Tuyển dụng",
        "giáo dục": "Giáo dục",
        "mới tốt nghiệp": "Giáo dục",
        "nội ngoại thất": "Thiết kế",
        "thẩm mỹ": "Thiết kế",
        "khách sạn": "Dịch vụ khách sạn",
        "tổ chức sự kiện": "Dịch vụ khách sạn",
        "nhà hàng": "Dịch vụ khách sạn",
        "du lịch": "Dịch vụ khách sạn",
        "luật": "Pháp lý",
        "bất động sản": "Bất động sản",
        "dệt may": "Dệt may / Da giày",
        "da giày": "Dệt may / Da giày",
        "bán lẻ": "Bán lẻ / Tiêu dùng",
        "chăm sóc sức khỏe": "Y tế / Chăm sóc sức khỏe",
        "dinh dưỡng": "Y tế / Chăm sóc sức khỏe",
        "chăm sóc cá nhân": "Y tế / Chăm sóc sức khỏe",
        "dược phẩm": "Dược",
        "thực phẩm & đồ uống": "Dịch vụ ăn uống",
        "hàng không": "Vận tải",
        "hàng hải": "Vận tải",
        "nông nghiệp": "Nông / Lâm / Ngư nghiệp",
        "thủy sản": "Nông / Lâm / Ngư nghiệp",
        "chăn nuôi": "Nông / Lâm / Ngư nghiệp",
        "thủy lợi": "Nông / Lâm / Ngư nghiệp",
        "địa chất": "Nông / Lâm / Ngư nghiệp",
        "thú y": "Nông / Lâm / Ngư nghiệp",
        "in ấn": "Nghệ thuật - Truyền thông / In ấn",
        "phi lợi nhuận": "Chính phủ / Phi lợi nhuận",
        "quy hoạch & quy định": "Chính phủ / Phi lợi nhuận",
        "lao động phổ thông": "Lao động phổ thông",
        "bảo vệ": "Lao động phổ thông",
        "an toàn lao động": "Lao động phổ thông",
        "thực tập": "Thực tập"
    }

    df = apply_sector_mapping(df, "nganh_nghe_clean", group)

    df["kinh_nghiem"] = df["exp_year"].apply(group_experience)

    X_text = vectorize_skills_word2vec(df, "ky_nang_combined")
    X_cats = encode_categorical(df, ["khu_vuc", "cap_bac_standardized", "kinh_nghiem"], "ohe_encoder.pkl")
    X_nganh = multilabel_nganh(df, "nganh_nghe_chuan_hoa", "mlb_nganh.pkl")

    X_all = hstack([csr_matrix(X_text), X_cats, X_nganh])
    np.save("/content/drive/MyDrive/CAPSTONE PROJECT/X_all_vector.npy", X_all.toarray())

    df[["ten_cong_viec", "link", "ky_nang_combined", "nganh_nghe_chuan_hoa"]].to_parquet(
        "/content/drive/MyDrive/CAPSTONE PROJECT/jobs_ML.parquet", index=False)

    print("Xử lý và lưu chuẩn hóa dữ liệu thành công.")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Xử lý và lưu chuẩn hóa dữ liệu thành công.
